In [1]:
import pandas as pd 
from nltk.tokenize import word_tokenize
import matplotlib.pyplot as plt 
from gensim.models import Word2Vec
import string
import numpy as np
from queue import PriorityQueue
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords 
import re
lemmatizer = WordNetLemmatizer() 

In [2]:
def get_dataset():
    file=open("Trec_microblog11.txt",encoding='utf-8')
    line=file.readline()
    nums=[]
    docs=[]
    while line:
      a = line.split('\t')
      num = a[0:1]
      doc = a[1:]
      nums.append(num)
      docs.append(doc)
      line = file.readline()
    return nums,docs

def train_docs():
  newdocs = []
  for doc in docs:
    doc[0] = re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0–9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0–9@:%_\+.~#?&//=]*)', " ", doc[0])
    doc[0] = re.sub(r'@([a-z]+) ([a-z]+),',' ', doc[0]) 
    punc = '~`!#$%^&*()_+-=|\';":/.,?><~·！@#￥%……&*（）——+-=“：’;.，?》《><{}``'
    doc[0] = re.sub(r"[%s]+" %punc, " ",doc[0]) 
    doc[0] = doc[0].replace('\t',' ')
    doc[0] = doc[0].replace('\xa0',' ')
    newdoc = doc[0].replace('\n','')
    newdoc = doc[0].lower() 
    newdocs.append(text_processing(newdoc))
  return newdocs

def text_processing(question):
    stopWords=set(stopwords.words('english'))
    every_question_word = list()
    sentence = []
    for word in word_tokenize(question):
      if (word not in stopWords):
        every_question_word.append(lemmatizer.lemmatize(word))
    return every_question_word

nums,docs = get_dataset()
newdocs = train_docs()

In [3]:
question_list = newdocs
word_in_q = {}

for x in question_list:
  for y in x:
    if y in word_in_q:
      word_in_q[y] +=1
    else:
      word_in_q[y] = 1

word_num = sum(word_in_q.values())
word_different = len(word_in_q.keys())

dictionary = dict(sorted(word_in_q.items(),key=lambda x:x[1],reverse=True))

In [4]:
# it will take long time like 3-5min, please wait for it
all_question_word = question_list
index_inverted = dict()
for question in question_list:
    for word in question:
        if word not in index_inverted:
            index_inverted.update({word: [question_list.index(question)]})
        else:
            index_inverted[word].append(question_list.index(question))

In [5]:
# it will take long time like 3-5min, please wait for it
model = Word2Vec(all_question_word,min_count=1,size = 200,iter = 30,sample=1e-3,sg=1,window = 5)
model.save("word2vec.model")
model = Word2Vec.load("word2vec.model")
model.train(all_question_word,total_examples=model.corpus_count, epochs=model.iter) 

<ipython-input-5-f86ad2a24cba>:5: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).
  model.train(all_question_word,total_examples=model.corpus_count, epochs=model.iter)


(17054520, 17369170)

In [6]:
def get_query():
    f=open("topics_MB1-49.txt",encoding='utf-8')
    line=f.readline()
    query=[]
    while line:
        thisquery=[]
        if ('<title>' in line):
          line=line.strip('<title>')
          line=line.strip('</title>\n')
          line = re.sub('",', " ", line)
          line = line.lower() 
          words=word_tokenize(line)
          for i in words:
            if i != "''" and i !='``' and i !="'" and i !=',':
              thisquery.append(i)
          query.append(thisquery)
        else:
          line = f.readline()
    return query
queries = get_query()

In [7]:
findnumslist = []
percs = []
from scipy import spatial

for real_question_word in queries:
  filter_index = set()
  real_question_word_exist = real_question_word[:] 
  for word in real_question_word:   
    if word not in index_inverted.keys():
      real_question_word_exist.remove(word)
    else:
      filter_index.update(index_inverted[word])
  part_question_word = list()
  for index in filter_index:
    one_question_word = all_question_word[index]
    part_question_word.append(one_question_word)
  siml_list = list()
  for question_word in part_question_word:
    siml=model.wv.n_similarity(real_question_word_exist,question_word)
    siml_list.append(siml)
  if len(siml_list)==0:
    findnumslist.append([])
    percs.append([])
  elif max(siml_list)>0:
    top_question = []
    for i in range(len(siml_list)):
        top_question.append((siml_list[i],part_question_word[i]))
    sorted_top_question = sorted(top_question)
    sorted_top_question.reverse()
    if len(sorted_top_question) > 1000:
        sorted_top_question = sorted_top_question[:1000] 
    findnums = []
    perc = []
    for top in sorted_top_question:
      per,find = top
      findnum = nums[newdocs.index(find)]
      findnums.append(findnum)
      perc.append(per)

    findnumslist.append(findnums)
    percs.append(perc)


In [8]:
def output():
  with open('outputword2vec.txt','w') as fa:
    for i in range(len(findnumslist)):
      if findnumslist[i] != []:
        for j in range(len(findnumslist[i])):
          out = str(i+1) + ' Q0 ' + str(findnumslist[i][j][0]) + ' ' + str(j+1) + ' ' + str(percs[i][j]) + ' query NO.' + str(j+1) + ''"\n"
          fa.write(out)
    fa.close()
output()